In [1]:
!pip install -U albumentations ultralytics optuna huggingface_hub
!git clone https://github.com/sathishkumar67/ADIS.git
!mv /kaggle/working/ADIS/* /kaggle/working/

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.0/66.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.6/290.6 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 949.8/949.8 kB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 632.7/632.7 kB 28.3 MB/s eta 0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.29.0
    Uninstalling huggingface-hub-0.29.0:
      Successfully uninstalled huggingface-hub-0.29.0
  Attempting uninstall: albucore
    Found existing installation: albucore 0.0.19
    Uninstalling albucore-0.0.19:
      Successfully uninstalled albucore-0.0.19
  Attempting uninstall: albumentations
    Found existing installation: albumentations 1.4.20
    Uninstalling albumentations-1.4.20:
      Successfully uninstall

In [2]:
REPO_ID = "pt-sk/ADIS" 
FILENAME_IN_REPO = "dataset.zip"
LOCAL_DIR = "/kaggle/working/"
TRAIN_PATH = f"{LOCAL_DIR}dataset/train"
VAL_PATH = f"{LOCAL_DIR}dataset/val"
TEST_PATH = f"{LOCAL_DIR}dataset/test"
DATASET_PATH = f"{LOCAL_DIR}{FILENAME_IN_REPO}"
REPO_TYPE = "dataset"
NUM_CLASSES = 10                                               
CLASSES = ['Cat', 'Cattle', 'Chicken', 'Deer', 'Dog', "Squirrel", 'Eagle', 'Goat', 'Rodents', 'Snake'] 
DATA_YAML_FILE = f"{LOCAL_DIR}/data.yaml"
MODEL_PATH = "yolo11n.pt"

In [3]:
from huggingface_hub import hf_hub_download
from utils import unzip_file
import os

# download the dataset and unzip it
hf_hub_download(repo_id=REPO_ID, filename=FILENAME_IN_REPO, repo_type=REPO_TYPE, local_dir=LOCAL_DIR)
unzip_file(DATASET_PATH, LOCAL_DIR)


# Get the number of CPU cores
num_cores = os.cpu_count()
print(f"Number of CPU cores: {num_cores}")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


dataset.zip:   0%|          | 0.00/9.57G [00:00<?, ?B/s]

Unzipping: 100%|██████████| 9.60G/9.60G [01:02<00:00, 154MB/s] 

Number of CPU cores: 4


In [4]:
# split paths for model
data_yaml = f"""
train: {TRAIN_PATH}
val: {VAL_PATH}
test: {TEST_PATH}

nc: {NUM_CLASSES}
names: {CLASSES}
"""

# write data yaml file
with open(DATA_YAML_FILE, "w") as file:
    file.write(data_yaml)
    print("data yaml file written!.............")

data yaml file written!.............


In [5]:
import optuna
from model import YOLO11Model

def objective(trial):
    
    # Define callback to report intermediate results
    def on_train_epoch_end(validation_loss, epoch):
        trial.report(validation_loss, step=epoch)  
        if trial.should_prune():
            raise optuna.TrialPruned()

    callbacks = {
        "on_train_epoch_end" : on_train_epoch_end
    }
    
    # Define hyperparameters using Optuna suggestions
    lr0 = trial.suggest_float("lr0", 1e-5, 1e-3, log=True)
    lrf = trial.suggest_float("lrf", 0.1, 1, log=True)
    weight_decay = trial.suggest_float("weight_decay", 0.0001, 0.01, log=True)
    warmup_momentum = trial.suggest_float("warmup_momentum", 0, 0.9)
    momentum = trial.suggest_float("momentum", 0.85, 0.99)
    
    CONFIG_DICT = {
    "task": "detect",
    "mode": "train",
    "bohb": True,
    "custom_callbacks": callbacks,
    "data": DATA_YAML_FILE,
    "batch": 120,
    "imgsz": 640,
    "save": True,
    "device": 0,
    "workers": num_cores,
    "pretrained": True,
    "optimizer": "AdamW",
    "seed": 42,
    "epochs": 50,
    "warmup_epochs": 5,
    "patience": 5}

    # Train YOLO model
    model = YOLO11Model(MODEL_PATH)
    results = model.train(**CONFIG_DICT, lr0=lr0, lrf=lrf, momentum=momentum, weight_decay=weight_decay, warmup_momentum=warmup_momentum)
    
    # Return validation mAP as the objective value
    return model.final_validation_loss

In [ ]:
NUM_TRIALS = 50
study = optuna.create_study(direction='minimize', 
                            sampler=optuna.samplers.TPESampler(), 
                            pruner=optuna.pruners.HyperbandPruner(),
                            study_name="yolo11_tuning",
                            load_if_exists=True)

study.optimize(objective, n_trials=NUM_TRIALS)

# import joblib
# joblib.dump(study, "optuna_study.pkl")

[I 2025-03-27 10:59:00,177] A new study created in memory with name: yolo11_tuning


100%|██████████| 5.35M/5.35M [00:00<00:00, 108MB/s]


Ultralytics 8.3.97 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
trainer: task=detect, mode=train, model=yolo11n.pt, data=/kaggle/working//data.yaml, epochs=50, time=None, patience=5, batch=120, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=4, project=None, name=train, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=42, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_bo

100%|██████████| 755k/755k [00:00<00:00, 31.3MB/s]


Overriding model.yaml nc=80 with nc=10

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  blocks.Conv                                  [3, 16, 3, 2]                 
  1                  -1  1      4672  blocks.Conv                                  [16, 32, 3, 2]                
  2                  -1  1      6640  blocks.C3k2                                  [32, 64, 1, False, 0.25]      
  3                  -1  1     36992  blocks.Conv                                  [64, 64, 3, 2]                
  4                  -1  1     26080  blocks.C3k2                                  [64, 128, 1, False, 0.25]     
  5                  -1  1    147712  blocks.Conv                                  [128, 128, 3, 2]              
  6                  -1  1     87040  blocks.C3k2                                  [128, 128, 1, True]           
  7                  -1  1    295424  blocks.Con

train: Scanning /kaggle/working/dataset/train... 20000 images, 4 backgrounds, 0 corrupt: 100%|██████████| 20000/20000 [00:17<00:00, 1142.95it/s]


train: New cache created: /kaggle/working/dataset/train.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Scanning /kaggle/working/dataset/val... 1400 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1400/1400 [00:01<00:00, 1097.56it/s]


val: New cache created: /kaggle/working/dataset/val.cache
Plotting labels to runs/detect/train/labels.jpg... 
optimizer: AdamW(lr=3.0399969144740867e-05, momentum=0.9585461980317896) with parameter groups 75 weight(decay=0.0), 82 weight(decay=0.001218057080331183), 81 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 4 dataloader workers
Logging results to runs/detect/train
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/50      15.4G      1.146      3.316      1.513        378        640:  43%|████▎     | 71/167 [01:31<02:09,  1.34s/it]

In [ ]:
# visualize plots

# from optuna.visualization import plot_optimization_history, plot_parallel_coordinate, plot_param_importances

# # After optimization
# plot_optimization_history(study).show()  # Trial history
# plot_parallel_coordinate(study).show()   # Hyperparameter interactions
# plot_param_importances(study).show()     # Importance of each hyperparameter

In [ ]:
# import torch
# import gc

# gc.collect()  # Run garbage collector
# torch.cuda.empty_cache()  # Free unused memory
# torch.cuda.memory_allocated(), torch.cuda.memory_reserved()
# torch.cuda.empty_cache()
# torch.cuda.ipc_collect()

In [ ]:
# from model import YOLO11Model
# import torch
# model = YOLO11Model(MODEL_PATH)
# results = model.train(task="detect", mode="train", data=DATA_YAML_FILE, epochs=1, lr0=1e-5, batch=80, imgsz=640, save=True, optimizer="AdamW", lrf=0.03, momentum=0.9, weight_decay=0.001, device=0)